In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Frequency analysis 
## Class Data()
This class takes a xarray.Dataset to initialise
After the class is initialised, it can be used to prepare the data
in this example, we extract the value form a melcc dataset and we filter on all catchments statirng with 0234- and catchemnt 051001
The function get_julian_day helps us defining our seasons dates which we then set using the attribute .season of our Data() DataSet

- [loading the dataset Cell](#data_load)
- [Catchement selection](#data_catchments)
- [Seasons](#data_seasons)
- [rm Seasons](#data_rmseasons)
- [Maximum](#data_getMaximum)

## Class Local()
- [Init](#fa_init)
- [View quantiles](#fa_qunatiles)






<a id='data_load'></a>
Load the dataset

In [44]:
import xarray as xr
import xhydro as xh
import numpy as np
import xdatasets as xd

In [45]:
ds = xd.Query(
    **{
        "datasets":{
            "deh":{
                "id" :["02*"],
                "regulated":["Natural"],
                "variables":["streamflow"],
            }
        }, "time":{"start": "1970-01-01", 
                   "minimum_duration":(15*365, 'd')},

  }
).data
ds

<xarray.Dataset>
Dimensions:        (id: 37, time: 19523, variable: 1, spatial_agg: 1,
                    timestep: 1, time_agg: 1, source: 1)
Coordinates: (12/15)
    drainage_area  (id) float32 1.09e+03 647.0 59.8 ... 227.0 2.163e+03 48.1
    end_date       (variable, id, spatial_agg, timestep, time_agg, source) datetime64[ns] ...
  * id             (id) object '020302' '020404' '020502' ... '024014' '024015'
    latitude       (id) float32 48.77 48.81 48.98 48.98 ... 46.05 46.2 46.18
    longitude      (id) float32 -64.52 -64.92 -64.43 ... -71.45 -72.1 -71.75
    name           (id) object 'Saint' 'York' ... 'Bécancour' 'Bourbon'
    ...             ...
  * spatial_agg    (spatial_agg) object 'watershed'
    start_date     (variable, id, spatial_agg, timestep, time_agg, source) datetime64[ns] ...
  * time           (time) datetime64[ns] 1970-01-01 1970-01-02 ... 2023-06-14
  * time_agg       (time_agg) object 'mean'
  * timestep       (timestep) object 'D'
  * variable       (variable) object 'streamflow'
Data variables:
    streamflow     (id, time, variable, spatial_agg, timestep, time_agg, source) float32 dask.array<chunksize=(1, 19523, 1, 1, 1, 1, 1), meta=np.ndarray>

<a id='data_catchments'></a>
Use the function <span style="color:lime">select_catchments</span>. to select the catchment, 
<br>this function can use a wildcard.
<br>This will create a new object Data
<br><span style="color:lime">.catchments</span>. is used to show the cathcments, 

In [46]:
donnees = xh.Data(ds)

In [47]:
catchment_list = ['02*','03*', '05*']
sub_set = donnees.select_catchments(catchment_list = catchment_list)

<a id='data_seasons'></a>
listing Data's seasons

In [48]:
a_beg = xh.get_julian_day(month = 9,day = 1)
a_end = xh.get_julian_day(month = 12,day = 1)

p_beg = xh.get_julian_day(month = 2,day = 11)
p_end = xh.get_julian_day(month = 6,day = 19)


#Passer dictionnaire
sub_set.season = ['Automne', a_beg, a_end]
sub_set.season = ['Spring', p_beg, p_end]
sub_set.season = ['Annuelle', 1, 365]

sub_set.get_seasons()


AttributeError: module 'xhydro' has no attribute 'get_julian_day'

<a id='data_rmseasons'></a>
Use rm_season to remove a season

In [ ]:
sub_set.rm_season('Annuelle')
sub_set.get_seasons()

La cellule suivante ne sert qu'a générer un dataset aléatoire avec des dates de début et de fin de crue. 
­<br>Normalement ces dates seront chargées ou déterminées visuellement pas l'utilisateur

In [ ]:
import random
# Generating random flood dates for all year and one catchment
m = np.empty((1, len(range(1910, 2023))), dtype=object)
for i in np.ndindex(m.shape): m[i] = [random.randint(70, 70), random.randint(139, 139)]


dates_ds = xr.Dataset()

dates_ds.coords['year'] = xr.DataArray(range(1910, 2023), dims=('year',))
dates_ds.coords['id'] = xr.DataArray(['051001'], dims=('id',))

dates_ds['value'] = xr.DataArray(m, dims=('id', 'year'))


In [ ]:
sub_set._season['Spring_ds'] = dates_ds

In [ ]:

mm1 = sub_set._get_max(tolerence=0.15, seasons=['Spring_ds'])
mm2 = sub_set._get_max(tolerence=0.15, seasons=['Spring'])
# TODO max are the same, but first and last years are mission from Spring_ds

In [ ]:
mm1.to_dataframe(name = 'Maximums').reset_index()[['id', 'season', 'year', 'start_date','end_date','Maximums']].dropna()

In [ ]:
mm2.to_dataframe(name = 'Maximums').reset_index()[['id', 'season', 'year', 'start_date','end_date','Maximums']].dropna()

On calcul les volume de crues à dates fixes

In [ ]:
vol = sub_set.calculate_volume(dates=[35, 36])
vol.volume.to_dataframe()

On calcul les volumes de crues avec un DataSet

In [ ]:
sub_set_example = sub_set.select_catchments(['051001'])
vol = sub_set_example.calculate_volume(dates=dates_ds)
vol.to_dataframe().dropna()


In [ ]:
vol.to_dataframe().dropna().reset_index()

<a id='data_getMaximum'></a>
Use get_maximum to get the maximums per season for selected catcment, if no period selected, anual maxmaximum will be fectch

In [ ]:
sub_set.get_maximum(tolerence=.85)

In [ ]:
sub_set.get_maximum(tolerence=0.15, seasons=['Spring'])

# CLass Local()

<a id='fa_init'></a>
Init local with a data Ds

In [ ]:
return_period = np.array([2, 5, 10, 20, 50, 100, 200, 1000, 2000, 10000])
dist_list = ['expon', 'gamma', 'genextreme', 'gennorm', 'gumbel_r', 'pearson3', 'weibull_min']

fa = xh.Local(data_ds = sub_set, return_period = return_period, dist_list = dist_list, tolerence = 0.15, seasons = ['Spring'], min_year = 15, vars_of_interest = ['max'])

In [ ]:
fa.analyse_max

In [ ]:
fa.view_values('max')

In [ ]:
fa.view_criterions('max')

In [ ]:
fa.view_quantiles('max')

In [ ]:
from xhydro.frequency_analysis.visualisation import *
plot_fa(fa, var='max', lang='fr', var_name='value')

In [ ]:
fa_calc = xh.Local(data_ds = mm2, return_period = return_period, dist_list = dist_list, tolerence = 0.15, seasons = ['Spring'], min_year = 15, vars_of_interest = ['max'], dates_vol=dates_ds, calculated = True)
fa_calc.view_values('max')

In [ ]:
fa.view_values('max')

In [ ]:
fa.view_criterions('max')

<a id='fa_qunatiles'></a>
view quantiles

In [ ]:
fa.view_quantiles('max')


In [ ]:
fa_calc.view_quantiles('max')
max

<a id='fa_criterions'></a>
view criterions